Now, after getting the final cleaned datasets, we can perform recommendation strategies. As per the available data, I have decided to perform two different methods to generate recommendation for a user.

Methods are as follows:

1) Top-N recommendations

2) Content based recommendations

### 2) Content based recommendations

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [2]:
df_tags = pd.read_csv('Data/tags.csv')

In [3]:
df_book_tags = pd.read_csv('Data/book_tags.csv')

In [4]:
df_book = pd.read_csv('Data/books.csv')

In [5]:
df_ratings = pd.read_csv('Data/ratings.csv')

In [6]:
df_genres = pd.read_csv('Data/genres.csv')

In [30]:
df_book = df_book[['goodreads_book_id','book_id','authors','original_title']]

In [ ]:
gen_list = df_genres.tag_name.tolist()

In [ ]:
df_tags = df_tags[df_tags.tag_name.isin(gen_list)]

In [ ]:
df_merged_tags = pd.merge(df_book_tags, df_tags, on = 'tag_id') 

In [ ]:
df_merged_tags

In [ ]:
df_full_info = pd.merge(df_book,df_merged_tags,on='book_id')

In [ ]:
df_full_combined_tags = df_full_info.groupby('goodreads_book_id')['tag_name'].apply(lambda x: "%s" % ' '.join(x)).reset_index()

In [ ]:
df_full_combined_tags = pd.merge(df_book,df_full_combined_tags,on='goodreads_book_id')

In [ ]:
df_full_combined_tags['authors'] = df_full_combined_tags['authors'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))

In [ ]:
df_full_combined_tags['authors'] = df_full_combined_tags['authors'].astype('str').apply(lambda x: str.lower(x.replace(",", " ")))

In [ ]:
df_full_combined_tags['Final_string'] = df_full_combined_tags['authors'] + ' ' + df_full_combined_tags['tag_name']

In [ ]:
df_full_combined_tags.Final_string[0]

In [ ]:
cnt_vectr = CountVectorizer(analyzer = 'word', ngram_range = (1, 2), min_df = 0, stop_words = 'english')

In [ ]:
token_Matrix =  cnt_vectr.fit_transform(df_full_combined_tags.Final_string)

In [ ]:
cos_sim = cosine_similarity(token_Matrix,token_Matrix)

In [ ]:
df_full_combined_tags = df_full_combined_tags.reset_index()
bookTitles = df_full_combined_tags['original_title']
indices = pd.Series(df_full_combined_tags.index, index = bookTitles)

In [ ]:
def MoreLikeThis(title):
    index = indices[title]
    similarityScore = list(enumerate(cos_sim[index]))
    similarityScore = sorted(similarityScore, key = lambda x: x[1], reverse = True)
    similarityScore = similarityScore[1:11]
    bookIndex = [i[0] for i in similarityScore]
    return bookTitles.iloc[bookIndex]

In [ ]:
MoreLikeThis('The Hunger Games')

In [ ]:
u2 = df_ratings[df_ratings.user_id==2]

In [ ]:
u2_blist = u2.book_id.tolist()

In [ ]:
u2_blist


In [ ]:
(df_book.goodreads_book_id==127).value_counts()

In [ ]:
(df_book.goodreads_book_id == u2_blist[2]).value_counts()

In [ ]:
df_book[df_book.goodreads_book_id == u2_blist[2]].original_title

In [ ]:
result = MoreLikeThis('Harry Potter and the Order of the Phoenix')

In [ ]:
res_list = result.index.tolist()

In [ ]:
u2_blist = set(u2_blist)
res_list = set(res_list)

In [ ]:
overlap = u2_blist & res_list
print(overlap)

In [ ]:
match = float(len(overlap)) / len(res_list) * 100
print(match)

In [ ]:
df_book.book_id.max()

In [ ]:
df_ratings.book_id.max()

In [ ]:
def accuracy(u_id):
    u = df_ratings[df_ratings.user_id==u_id]
    
    u_blist = u.book_id.tolist()
    
    for i in u_blist:
        print(i)
        for j in df_book.goodreads_book_id:
            print(j)
            if (i == j):
                title = df_book[df_book.goodreads_book_id==i].original_title
                print('----')
            else:
                print('goint to next')
                print('========')
            
            
        
                            
    result = MoreLikeThis(title)
    
    res_list = result.index.tolist()
    u_blist = set(u_blist)
    
    res_list = set(res_list)
   
    overlap = u_blist & res_list
   
    match = float(len(overlap)) / len(res_list) * 100
    print(match)
    
    
    
    